# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
from pprint import pprint

# Import API key
from config import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
cities_df = pd.read_csv("../WeatherPy/cities.csv", index_col = [0])
cities_df

,City,Country,Datetime,Latitude,Longitude,Max temperature (c),Humidity (%),Cloudiness (%),Wind Speed (mps)
0,new norfolk,AU,1635152870,-42.7826,147.0587,10.51,70,70,1.34
1,bilma,NE,1635152871,18.6853,12.9164,33.02,12,5,4.28
2,mount isa,AU,1635152581,-20.7333,139.5000,29.87,30,40,3.09
3,batemans bay,AU,1635152872,-35.7167,150.1833,15.05,65,35,3.45
4,west monroe,US,1635152873,32.5185,-92.1476,20.40,95,1,1.54
...,...,...,...,...,...,...,...,...,...
685,axim,GH,1635153540,4.8699,-2.2405,27.71,76,75,3.53
686,ternate,ID,1635153443,0.8000,127.4000,28.71,72,59,1.73
687,dharchula,IN,1635153542,29.8500,80.5333,22.13,43,0,2.60
688,zhezkazgan,KZ,1635153542,47.8043,67.7144,13.43,30,100,4.13


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [3]:
#configure gmaps
import gmaps
gmaps.configure(api_key = g_key)

In [4]:
#Add locations to list and convert humidity to float for weighting
locations = cities_df[["Latitude", "Longitude"]]

humidity = cities_df["Humidity (%)"].astype(float)

In [5]:
# Plot Heatmap
fig = gmaps.figure()

# Create heat layer
heat_layer = gmaps.heatmap_layer(locations, weights=humidity, 
                                 dissipating=False, max_intensity=humidity.max(),
                                 point_radius=1)


# Add layer
fig.add_layer(heat_layer)

# Display figure
fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [12]:
best_cities_df = cities_df.loc[(cities_df["Max temperature (c)"] > 20) 
                               & (cities_df["Max temperature (c)"] < 35) 
                               & (cities_df["Cloudiness (%)"] == 0)
                               & (cities_df["Humidity (%)"] < 18)].reset_index(drop = True)
best_cities_df

,City,Country,Datetime,Latitude,Longitude,Max temperature (c),Humidity (%),Cloudiness (%),Wind Speed (mps)
0,abadan,IR,1635152662,30.3392,48.3043,31.97,11,0,5.70
1,riyadh,SA,1635152467,24.6877,46.7219,33.08,11,0,2.41
2,adrar,MR,1635153029,20.5022,-10.0711,29.48,16,0,3.94
3,nushki,PK,1635152827,29.5542,66.0215,26.42,9,0,6.51
4,kharan,PK,1635153115,28.5833,65.4167,28.41,10,0,4.01
5,warmbad,NaN,1635153162,-28.4500,18.7333,24.25,14,0,5.41
6,siirt,TR,1635153247,37.9326,41.9403,22.16,17,0,1.54
7,bafq,IR,1635153512,31.6128,55.4107,23.47,13,0,0.02


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [13]:
best_cities_df["Hotel Name"] = " "
best_cities_df

,City,Country,Datetime,Latitude,Longitude,Max temperature (c),Humidity (%),Cloudiness (%),Wind Speed (mps),Hotel Name
0,abadan,IR,1635152662,30.3392,48.3043,31.97,11,0,5.70,
1,riyadh,SA,1635152467,24.6877,46.7219,33.08,11,0,2.41,
2,adrar,MR,1635153029,20.5022,-10.0711,29.48,16,0,3.94,
3,nushki,PK,1635152827,29.5542,66.0215,26.42,9,0,6.51,
4,kharan,PK,1635153115,28.5833,65.4167,28.41,10,0,4.01,
5,warmbad,NaN,1635153162,-28.4500,18.7333,24.25,14,0,5.41,
6,siirt,TR,1635153247,37.9326,41.9403,22.16,17,0,1.54,
7,bafq,IR,1635153512,31.6128,55.4107,23.47,13,0,0.02,


In [14]:
target_search = "Hotel"
target_radius = 5000
target_type = "hotel"
    
for index, row in best_cities_df.iterrows():
    try:
        lat = row[3]
        lng = row[4]
        params = {
            "location": (f"{lat},{lng}"),
            "keyword": target_search,
            "radius": target_radius,
            "type": target_type,
            "key": g_key
        }
        base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
        response = requests.get(base_url, params = params).json()
        hotel_name = (response["results"][0]["name"])
        best_cities_df.loc[index,"Hotel Name"] = hotel_name
    except IndexError:
        print(f"Hotel could not be found near city: {row[0]}")
        pass

Hotel could not be found near city: adrar


In [15]:
#remove rows where no hotel could be found
hotel_df = best_cities_df.loc[best_cities_df["Hotel Name"] != " "].reset_index(drop=True)
hotel_df

,City,Country,Datetime,Latitude,Longitude,Max temperature (c),Humidity (%),Cloudiness (%),Wind Speed (mps),Hotel Name
0,abadan,IR,1635152662,30.3392,48.3043,31.97,11,0,5.70,هتل و رستوران جم
1,riyadh,SA,1635152467,24.6877,46.7219,33.08,11,0,2.41,"Radisson Blu Hotel, Riyadh"
2,nushki,PK,1635152827,29.5542,66.0215,26.42,9,0,6.51,Nabi Bux Hotel
3,kharan,PK,1635153115,28.5833,65.4167,28.41,10,0,4.01,Mansoor Hotel & Restaurant
4,warmbad,NaN,1635153162,-28.4500,18.7333,24.25,14,0,5.41,Warmbad Museum
5,siirt,TR,1635153247,37.9326,41.9403,22.16,17,0,1.54,Barden Otel
6,bafq,IR,1635153512,31.6128,55.4107,23.47,13,0,0.02,Alamdar Hotel


In [16]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Latitude", "Longitude"]]

In [17]:
#Set marker locations
marker_locations = hotel_df[["Latitude", "Longitude"]]
locations = cities_df[["Latitude", "Longitude"]]
humidity = cities_df["Humidity (%)"].astype(float)

# Plot Heatmap
fig = gmaps.figure()

#Add marker layer
markers = gmaps.marker_layer(marker_locations, info_box_content = hotel_info)
fig.add_layer(markers)

#Add heat layer
heat_layer = gmaps.heatmap_layer(locations, weights=humidity, 
                                 dissipating=False, max_intensity=humidity.max(),
                                 point_radius=1)
fig.add_layer(heat_layer)

# Display figure
fig

Figure(layout=FigureLayout(height='420px'))